In [111]:
import os
import functools

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import spacy
from cached_property import cached_property

In [11]:
nlp = spacy.load(
    "en_core_web_lg",
    disable=["tagger", "ner", "textcat"]
)

In [2]:
articles = pd.read_parquet("data/articles-processed.parquet.gzip")

In [ ]:
class DiscriminatorNet(nn.Module):
    def __init__(self):
        super(DiscriminatorNet, self).__init__()

    def forward(self, hidden_state):
        """
        The forward pass for the network
        
        hidden_state : tensor (batch_num, hidden_size)
        
        returns         : tensor (batch_num, 1)
        """
        
        return hidden_state

In [91]:
class SummarizeNet(nn.Module):
    def __init__(self, hidden_size):
        super(SummarizeNet, self).__init__()
        
        self.hidden_size = hidden_size

    def forward(self, word_embeddings, generate="sentence"):
        """
        The forward pass for the network
        
        word_embeddings : tensor (batch_num, max_seq_len, vocab_len)
        
        returns         : tuple (
                            tensor (batch_num, max_seq_len, vocab_len),
                            tensor (batch_num, hidden_size)
                          )
        
        First tensor in the returning tuple is a probability over the vocabulary
        for each sequence position
        
        The second tensor is an encoder's hidden state 
        """
        
        return x

In [21]:
class ArticlesDataset(Dataset):
    def __init__(self, dataframe, mode, transforms=[]):
        self.data = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data = self.data.iloc[idx, :]

        for transform in self.transforms:
            data = transform(data)

        return data

In [ ]:
class WordsToVectors(object):
    def __init__(self, nlp):
        self.nlp = nlp
        
    def __call__(self, sample):
        raise NotImplementedError

In [ ]:
class AddNoiseToEmbeddings(object):
    def __init__(self, probability_of_mask_for_word, probability_of_masking_for_sample):
        self.probability_of_mask_for_word = probability_of_mask_for_word
        self.probability_of_masking_for_sample = probability_of_masking_for_sample
        
    def __call__(self, sample):
        raise NotImplementedError

In [ ]:
class ArticlesBatch:
    def __init__(self, data, id):
        self.data = data
        self.id = id
        
    def __getattr__(self, name):
        if name in self.data:
            return self.data[name]
        else:
            raise AttributeError(f"Attribute missing: {name}")

In [ ]:
class BaseTrainer:
    def __init__(self, name, nlp, summarize_model, discriminate_model, dataframe,
                 batch_size, update_every, save_every, loader_workers,
                 probability_of_mask_for_word, probability_of_masking_for_sample,
                 lambda_article, lambda_sentence):
        self.name = name
        
        self.datasets = {
            "train": ArticlesDataset(
                dataframe,
                "train",
                transforms=[
                    WordsToVectors(nlp),
                    AddNoiseToEmbeddings(
                        probability_of_mask_for_word,
                        probability_of_masking_for_sample
                    )
                ]
            ),
            "test":  ArticlesDataset(
                dataframe,
                "test",
                transforms=[
                    WordsToVectors(nlp)
                ]
            ),
            "eval":  ArticlesDataset(
                dataframe,
                "eval",
                transforms=[
                    WordsToVectors(nlp)
                ]
            )
        }
        
        self.batch_size = batch_size
        self.update_every = update_every
        self.save_every = save_every
        self.loader_workers = loader_workers
        
        self.summarize_model = summarize_model
        self.discriminate_model = discriminate_model
        
        self.lambda_article = lambda_article
        self.lambda_sentence = lambda_sentence
        
        self.current_batch_id = 0
        
    @property
    def models(self):
        return self.summarize_model, self.discriminate_model
    
    def save(self):
        checkpoint_path = f"checkpoints/{self.name}/batch-#{self.current_batch_id}"
        os.makedirs(checkpoint_path, exist_ok=True)
        
        torch.save(
            {
                'current_batch_id': self.current_batch_id,
                'batch_size': self.batch_size,
                'update_every': self.update_every,
                'save_every': self.save_every,
                'lambda_article': self.lambda_article,
                'lambda_sentence': self.lambda_sentence,
                'summarize_model_state': self.summarize_model.state_dict(),
                'discriminate_model_state': self.discriminate_model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict()
            },
            f"{checkpoint_path}/state.pth"
        )
        
    def load(name, dataframe):
        raise NotImplementedError
    
    def batches(self, mode):
        start_id = self.current_batch_id
        
        while True:
            loader = DataLoader(
                self.datasets[mode],
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=self.loader_workers
            )

            for ix, data in enumerate(loader):
                self.current_batch_id += ix
                
                yield(
                    ArticlesBatch(
                        data,
                        id=self.current_batch_id
                    )
                )
        
    
    def batch_loss(self, batch):
        raise NotImplementedError
        
    def after_update(self, batch, loss_sum):
        pass
    
    def train(self):
        batches = self.batches("train")
        loss_sum = 0
        
        for batch in batches:
            loss = self.batch_loss(batch) / (self.update_every * self.batch_size)
            
            loss.backward()
            loss_sum += loss
            
            # we're doing the accumulated gradients trick to get the gradients variance
            # down while being able to use commodity GPU:
            if batch.id % self.update_every == 0:
                self.optimizer.step()
                self.optimizer.zero_grad()
                
                self.after_update(batch, loss_sum)
                
                loss_sum = 0
    
    def test(self):
        raise NotImplementedError
    
    def evaluate(self):
        raise NotImplementedError

In [ ]:
class Trainer(BaseTrainer):
    def __init__(self, name, nlp, summarize_model, discriminate_model, dataframe,
                 batch_size, update_every, save_every, loader_workers,
                 probability_of_mask_for_word, probability_of_masking_for_sample,
                 lambda_article, lambda_sentence):
        super().__init__(
            name, nlp,
            summarize_model, discriminate_model, dataframe,
            batch_size, update_every, save_every, loader_workers,
            lambda_article, lambda_sentence
        )
        
    def compute_loss(self, articles_word_embeddings, orig_articles_word_embeddings,
                     sentences_word_embeddings, orig_sentences_word_embeddings,
                     sentences_numbers_in_articles,
                     discriminate_articles_probs,
                     discriminate_sentences_probs
                    ):
        articles_loss = F.cosine_embedding_loss(
            articles_word_embeddings,
            orig_articles_word_embeddings,
            torch.ones(articles_word_embeddings.shape[0])
        )
        
        sentences_loss = F.cosine_embedding_loss(
            sentences_word_embeddings,
            orig_sentences_word_embeddings,
            torch.ones(articles_word_embeddings.shape[0])
        ) / sentences_numbers_in_articles
        
        discriminator_articles_loss = F.binary_cross_entropy(
            discriminate_articles_probs,
            torch.zeros_like(discriminate_articles_probs)
        )
        
        discriminator_sentences_loss = F.binary_cross_entropy(
            discriminate_sentences_probs,
            torch.zeros_like(discriminate_sentences_probs)
        )
        
        return (articles_loss * self.lambda_article).sum(dim=0) +
               (sentences_loss * self.lambda_sentence).sum(dim=0) +
               discriminator_articles_loss +
               discriminator_sentences_loss
        

    def batch_loss(self, batch):
        # article -> article (de-noising)
        articles_word_embeddings, articles_state = self.summarize_model(
            batch.articles_noisy_word_embeddings,
            generate="article"
        )

        # sentence -> sentence (de-noising)
        sentences_word_embeddings, sentences_state = self.summarize_model(
            batch.sentences_noisy_word_embeddings,
            generate="sentence"
        )

        # the discriminator guessing which mode (article or sentence) was one state
        # created for to deal with the "segregation" problem described in the paper:
        discriminate_articles_probs = self.discriminate_model(articles_state)
        discriminate_sentences_probs = self.discriminate_model(sentences_state)

        # we're diverging from the article here by outputting the word embeddings
        # instead of the probabilities for each word in a vocabulary
        # our loss function is using the cosine embedding loss coupled with
        # the discriminator loss:
        return self.compute_loss(
            articles_word_embeddings,
            batch.articles_word_embeddings,

            sentences_word_embeddings,
            batch.sentences_word_embeddings,
            batch.sentences_numbers_in_articles,

            discriminate_articles_probs,
            discriminate_sentences_probs
        )

In [ ]:
trainer = Trainer(articles, 8)

for epoch in trainer.train():
    pass

In [23]:
dataset = ArticlesDataset(
    articles,
    mode="train"
)

In [29]:
print(SummarizeNet())

SummarizeNet()


In [25]:
next(iter(dataset))

headline            \nGet a bachelor’s degree.,\nEnroll in a studi...
text                 It is possible to become a VFX artist without...
normalized_title                         HowtoBeaVisualEffectsArtist1
set                                                             train
Name: 2, dtype: object

In [20]:
# for i in [t for t in list(nlp("Lorem ipsum dolor sit amet, consectetur adipiscing elit. YYy yyyy").sents)[0]]:
#     print(f"=> {i.vector}")